In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

#### Creating sigmoid function
formula: σ(x) = 1 / (1 + e^(-x))
 

In [3]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))



### loss function:
for this log los function is used(better for probabilistic values gradient descent):


Log Loss = -[y * log(p) + (1 - y) * log(1 - p)]


uses epilson to avoid error of log(0) and log(1)
epsilon = 1e-15


y_predicted_new = [max(i, epsilon) for i in y_predicted]: This loop goes through each predicted probability in the array y_predicted. If the predicted probability is smaller than epsilon, it's replaced with epsilon. This prevents taking the logarithm of very small probabilities, which could result in numerical issues (like getting negative infinity).

y_predicted_new = [min(i, 1 - epsilon) for i in y_predicted_new]: Similarly, this loop ensures that the predicted probabilities are not exactly 1 by replacing values greater than or equal to 1 - epsilon with 1 - epsilon. Again, this is to avoid taking the logarithm of very small numbers.


In [4]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

Negative Sign (-): Many optimization algorithms are designed to minimize a function. In the case of log loss, minimizing it corresponds to finding the best parameters that make the predicted probabilities match the true labels as closely as possible. However, since these optimization algorithms aim to minimize a function, it's common to use the negative of the function you want to minimize. This is why you see the negative sign in the formula.

Mean (Average): The mean (average) is taken because you are typically dealing with a dataset of multiple examples. You want to measure the overall performance of your model across all these examples. The mean ensures that you're not just looking at the performance on individual examples but considering the overall behavior of the model.

## Gradient descent

The learning rate (rate) is used to scale the gradients before subtracting them from the current parameter values. This scaling determines the size of the step taken in the direction that reduces the loss.

w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 


w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 



np.dot(), it calculates the dot product, which involves both element-wise multiplication and summation of the results.

In [8]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [9]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000, 0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:0.6812647787377568
Epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.29775789977965383, loss:0.6591474252715025
Epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.37150947240035115, loss:0.6431523291301916
Epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.4350664302689159, loss:0.6316873063379158
Epoch:5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.4899449005893882, loss:0.6234717079975919
Epoch:6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522855, loss:0.6175321183044205
Epoch:7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894964, loss:0.6131591858705934
Epoch:8, w1:0.9361540784567943, w2:0.7965748796787705, bias:-0.6155315088627656, loss:0.6098518179750948
Epoch:9, w1:0.9434791243557358, w2:0.7973647616854131, b

Epoch:86, w1:2.0096769928183367, w2:1.2581405625993878, bias:-1.554409536192387, loss:0.5461130291389041
Epoch:87, w1:2.023445915002755, w2:1.2610290989843815, bias:-1.5622915600184968, loss:0.545590563023641
Epoch:88, w1:2.037196382383489, w2:1.2638651346193672, bias:-1.5701263314476384, loss:0.5450712351014357
Epoch:89, w1:2.0509282490633733, w2:1.2666497026227028, bias:-1.577914553678509, loss:0.5445549975424178
Epoch:90, w1:2.0646413730163813, w2:1.2693838150830055, bias:-1.585656916406187, loss:0.544041803992791
Epoch:91, w1:2.0783356160084976, w2:1.2720684635220503, bias:-1.5933540961140944, loss:0.5435316095193146
Epoch:92, w1:2.092010843520487, w2:1.2747046193465692, bias:-1.6010067563586592, loss:0.5430243705560022
Epoch:93, w1:2.1056669246724873, w2:1.2772932342892063, bias:-1.6086155480469115, loss:0.5425200448529465
Epoch:94, w1:2.1193037321503643, w2:1.2798352408388918, bias:-1.6161811097072503, loss:0.5420185914271817
Epoch:95, w1:2.1329211421337653, w2:1.282331552660889,

Epoch:168, w1:3.069624635705023, w2:1.384948600792189, bias:-2.0885095712955053, loss:0.5110525650389492
Epoch:169, w1:3.081636942597884, w2:1.385683334629563, bias:-2.0940235982327984, loss:0.5107009675295656
Epoch:170, w1:3.093626900549122, w2:1.386407706162234, bias:-2.099520390044595, loss:0.5103508590024443
Epoch:171, w1:3.105594521513913, w2:1.387121922726688, bias:-2.105000110654235, loss:0.5100022310118231
Epoch:172, w1:3.117539818203424, w2:1.387826187748753, bias:-2.1104629213478976, loss:0.5096550752083746
Epoch:173, w1:3.1294628040692873, w2:1.3885207008179221, bias:-2.115908980823003, loss:0.5093093833367357
Epoch:174, w1:3.141363493288346, w2:1.3892056577601883, bias:-2.1213384452356676, loss:0.5089651472331199
Epoch:175, w1:3.1532419007476733, w2:1.389881250709425, bias:-2.1267514682472273, loss:0.5086223588230152
Epoch:176, w1:3.165098042029856, w2:1.390547668177342, bias:-2.1321482010698514, loss:0.5082810101189588
Epoch:177, w1:3.176931933398537, w2:1.3912050951220503

Epoch:256, w1:4.043744846294124, w2:1.4258419999584127, bias:-2.5210915122705426, loss:0.4850450142879176
Epoch:257, w1:4.053891525923789, w2:1.4261580936914633, bias:-2.525511176337824, loss:0.48479911427360317
Epoch:258, w1:4.064018585775913, w2:1.4264727355143816, bias:-2.529921626552404, loss:0.4845541776498612
Epoch:259, w1:4.074126066330109, w2:1.4267859632096573, bias:-2.534322909307031, loss:0.48431020001040465
Epoch:260, w1:4.084214008125393, w2:1.4270978138051291, bias:-2.5387150704342933, loss:0.48406717697273477
Epoch:261, w1:4.094282451756979, w2:1.4274083235878308, bias:-2.5430981552158864, loss:0.48382510417794483
Epoch:262, w1:4.104331437873139, w2:1.4277175281175865, bias:-2.5474722083917123, loss:0.4835839772905306
Epoch:263, w1:4.114361007172114, w2:1.428025462240357, bias:-2.5518372741688182, loss:0.4833437919982008
Epoch:264, w1:4.124371200399091, w2:1.4283321601013441, bias:-2.5561933962301766, loss:0.4831045440116945
Epoch:265, w1:4.134362058343228, w2:1.42863765

Epoch:352, w1:4.9341903172592385, w2:1.453116271096432, bias:-2.9085747567736684, loss:0.4652716491080398
Epoch:353, w1:4.942638809680464, w2:1.45339150249431, bias:-2.91226433607134, loss:0.4651010456723471
Epoch:354, w1:4.95107153085077, w2:1.453666829143015, bias:-2.9159475189195407, loss:0.4649310709621243
Epoch:355, w1:4.959488518337886, w2:1.453942254870665, bias:-2.919624323972772, loss:0.46476172226543927
Epoch:356, w1:4.967889809631988, w2:1.4542177833837202, bias:-2.9232947697566276, loss:0.4645929968835612
Epoch:357, w1:4.976275442145338, w2:1.4544934182693938, bias:-2.9269588746695274, loss:0.46442489213089116
Epoch:358, w1:4.984645453211936, w2:1.4547691629980217, bias:-2.93061665698442, loss:0.464257405334892
Epoch:359, w1:4.9929998800871696, w2:1.4550450209253871, bias:-2.934268134850457, loss:0.46409053383601956
Epoch:360, w1:5.001338759947489, w2:1.4553209952950035, bias:-2.9379133262946424, loss:0.4639242749876548
Epoch:361, w1:5.00966212989009, w2:1.4555970892403576,

(5.051047623653049, 1.4569794548473887, -2.9596534546250037)